In [ ]:
TPU_ENVIRON=False # find out the way to detect it 
# %%capture
if TPU_ENVIRON: # and kaggle
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version 1.13 --apt-packages libomp5 libopenblas-dev
    !pip install --upgrade timm numpy wandb torchmetrics pytorch-lightning typing-extenson==4.4.0

    # local imports to keep things neat
    import sys
    import importlib
    import typing_extensions
    import tensorboard
    importlib.reload(tensorboard)
    import pytorch_lightning as pl
    pl.loggers.tensorboard_TENSORBOARD_AVAILABLE = True


Each TPU core uses two-dimensional 8 X 128 vector registers for processing matrix multiplications. In general, your batch size should be evenly divisible by 8 or 128.

In [ ]:
!nvidia-smi

### Environment/Dataset configuration

## TLDR
`kaggle.json` - Kaggle API key

wandb - if enabled enter api key in notebook (need to change - troublesome)

## Details
Basicaly 3 options:
* Kaggle environement
* Google colab environment
* Other

When running on kaggle, kaggle datasets are available since runtime is ready.
Otherwise one have to download dataset - downloading from Kaggle is good option.
**In order to download kaggle dataset one need his own API key**


Original Kaggle DICOM dataset ~300GB needs preprocessing.
`kaggle competitions download -c rsna-breast-cancer-detection`

Currently I use community-processed or self-processed training data.
Kaggle supports creating own private datasets up to 20GB each one.

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except ImportError:
    IS_KAGGLE = False

In [ ]:
import os
import sys
import os.path
from pathlib import Path

In [ ]:
# DATASET_URI = "awsaf49/rsna-bcd-roi-1024x512-png-v2-dataset"
# DATASET_PATH_SUFFIX = ""
DATASET_URI = "rifti86793/rsna-16bit-512"
DATASET_PATH_SUFFIX = "rsnaroi51216bit"
DATASET_DIR = os.path.split(DATASET_URI)[1]
DATASETS_LOCAL_REPO = "/kaggle/input/" if IS_KAGGLE else os.path.expanduser("~/rsna-breast")
DATASET_PATH_START = os.path.join(DATASETS_LOCAL_REPO, DATASET_DIR)
DATASET_PATH = os.path.join(DATASET_PATH_START, DATASET_PATH_SUFFIX)

In [ ]:
# TODO add wandb api key
if not os.path.exists(DATASET_PATH_START):
    try:
        from google.colab import files
        if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
            files.upload()
            #2. Series of commands to set-up for download
            !ls -lha kaggle.json
            !mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
            !cp kaggle.json ~/.kaggle/ # move the key to the folder
            #3. giving rw access (if 401-nathorized)
            !chmod 600 ~/.kaggle/kaggle.json
    except ImportError:
        pass
    if not IS_KAGGLE:
        !pip install -q kaggle # installing the kaggle package
        if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")) and os.path.exists("kaggle.json"):
            !mkdir -p ~/kaggle/
            !cp kaggle.json ~/.kaggle/ # move the key to the folder
            #3. giving rw access (if 401-nathorized)
            !chmod 600 ~/.kaggle/kaggle.json
        !kaggle datasets download -d {DATASET_URI} -p {DATASETS_LOCAL_REPO}
        !unzip -q {DATASET_PATH_START}.zip -d {DATASET_PATH_START}
        !rm {DATASET_PATH_START}.zip

In [ ]:
if IS_KAGGLE:
    sys.path.append("../input/timm-pytorch-image-models")

## Installation and Imports

In [ ]:
# %%capture
if not TPU_ENVIRON:
    !pip install --upgrade timm wandb torchmetrics pytorch-lightning albumentations opencv-python typing-extensions

In [ ]:
import os
from pathlib import Path
import sys
import cv2
import glob
import numpy as np
import pandas as pd
import math
import enum
import matplotlib.pyplot as plt
from typing import Iterator, Iterable, Optional, Sequence, List, TypeVar, Generic, Sized, Union

import wandb

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, Sampler

import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
import warnings

import sklearn.model_selection

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize, Affine,
    RandomGamma,
    ImageOnlyTransform, ToFloat
)
import albumentations.augmentations.transforms
import albumentations.augmentations.utils
import albumentations.augmentations.functional
from albumentations.pytorch import ToTensorV2

# import warnings
# warnings.simplefilter('ignore')


In [ ]:
%matplotlib inline

## Config File and Wandb

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class WandbMode:
    OFFLINE = "offline"
    ONLINE = "online"
    DISABLED = "disabled"
    
    
class WandbConfig:
    # change to disabled/offline if not needed
    mode = WandbMode.ONLINE
    project = "rsna-breast"
    name = None
    save_dir = None
    log_model="all"
    watch = "all"
    watch_log_freq = 100
    

class ModelCheckpointSource(str, enum.Enum):
    NO_CHECKPOINT="no_checkpoint"
    WANDB="wandb"
    LOCAL="local"

class Config:
    train_bs=16 if not TPU_ENVIRON else 1024
    valid_bs=16 if not TPU_ENVIRON else 1024
    model_name="tf_efficientnetv2_s"
    model_use_pretrained=True
    model_params = {
      "drop_rate": 0.2,
      "drop_path_rate": 0.2
    }
    
    model_checkpoint_source = ModelCheckpointSource.NO_CHECKPOINT
    model_checkpoint_uri = "checkpoints/last-v13.ckpt"
    # True, to use Config params instead of checkpoints
    overwrite_checkpoint_hparams=True
    
    train_imgs_path = f"{DATASET_PATH}/train_images"
    
    # True if image data folders have structure <patient_id>/<image_id>.png
    # False if image data have structure <patient_id>_<image_id>.png
    data_has_patient_folder_sturcture = True
    
    # MADE error in my dataset  - need to use kaggles
    #train_csv_path = "/kaggle/input/rsna-breast-cancer-detection/train.csv"
    train_csv_path = f"{DATASET_PATH}/train.csv"
    
    test_imgs_path = f"{DATASET_PATH}/test_images/"
    test_csv_path = f"{DATASET_PATH}/test.csv"
    
    accelerator="auto"   # auto, gpu or cpu
    # TODO - parallel computations
    num_devices=1 if not TPU_ENVIRON else 8   

    resize_aspect_ratio = 1.0
    use_pretrained=True
    epochs_count = 20
    splits_count = 1
    warmup_lr = 1e-5
    warmup_epochs = 0
    lr=8e-4
    t_max= 4
    min_lr= 4e-4
    weight_decay=5e-3
    # weight of cancer examples in loss function
    pos_weight = 1
    # upsample cancers to percent
    weight_random_sampler = True
    positive_upsample_to_percent = 0.10
    # mixed-precision
    precision=16 if torch.cuda.is_available() else 32
    # value to pad smaller image in batch to the size of larger
    batch_pad_value = 0
    
    
    # dataloaders
    dataloader_workers_count = 4 if not TPU_ENVIRON else 64  
    # aux_targets = ['BIRADS'] # not used
    
    debug=False
    debug_data_use_only_percent=0.001
    

def config_class_to_dict(config_class):
    # add recursion if needed
    return { k:v for k,v in Config.__dict__.items() if not k.startswith("__") }

In [ ]:
print(Config.train_imgs_path)
print(Path(Config.train_imgs_path).exists())

In [ ]:
wandb_logger = None
wandb_artifact = None
wandb_artifact_dir = None
if WandbConfig.mode != WandbMode.DISABLED:
    wb_key = None
    if IS_KAGGLE:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        wb_key = user_secrets.get_secret("WANDB_API_KEY")
    else:
        wb_key = os.environ.get("WANDB_API_KEY")
    # wandb should get key fron environment
    wandb.login(key=wb_key)

    wandb_logger = WandbLogger(
        project=WandbConfig.project,
        name = WandbConfig.name,
        offline = True if WandbConfig.mode == WandbMode.OFFLINE else False,
        save_dir = WandbConfig.save_dir,
        log_model = WandbConfig.log_model,
        group='vision',
        job_type='train',
        config=config_class_to_dict(Config)
    )


In [ ]:
checkpoint_path = None
if Config.model_checkpoint_source==ModelCheckpointSource.LOCAL:
    checkpoint_path = Path(Config.model_checkpoint_uri)
if WandbConfig.mode == WandbMode.ONLINE and Config.model_checkpoint_source==ModelCheckpointSource.WANDB:
    artifact = wandb.run.use_artifact(
        Config.model_checkpoint_uri, type="model")
    artifact_dir = artifact.download()
    checkpoint_path = Path(artifact_dir) / "model.ckpt"

In [ ]:
# not used, but efficientnet v2 do not use weight decay
# for some parameters
def add_weight_decay(model, weight_decay=1e-5, skip_list=()):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if len(param.shape) == 1 or np.any([v in name.lower()  for v in skip_list]):
            # print(name, 'no decay')
            no_decay.append(param)
        else:
            # print(name, 'decay')
            decay.append(param)
    return [
        {'params': no_decay, 'weight_decay': 0.},
        {'params': decay, 'weight_decay': weight_decay}]

In [ ]:
def create_test_opt_and_scheduler():
    opt = torch.optim.SGD([torch.tensor(1)], lr=Config.lr)
    scheduler1 = torch.optim.lr_scheduler.ConstantLR(opt,
                                                     factor=Config.warmup_lr/Config.lr,
                                                     total_iters=20)
    scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(
                opt, 
                T_max=Config.t_max,
                eta_min=Config.min_lr
                )
    scheduler = torch.optim.lr_scheduler.SequentialLR(opt,
                                                      schedulers=[scheduler1, scheduler2],
                                                      milestones=[Config.warmup_epochs])
    return opt, scheduler

In [ ]:


def visualise_scheduler(opt, scheduler, steps=Config.epochs_count, lr=1):
    lrs = []
    for _ in range(100):
        opt.step()
        lrs.append(scheduler.get_last_lr())
        scheduler.step()
    
    fig, ax = plt.subplots()
    ax.plot(lrs)
    fig.show()
opt, scheduler = create_test_opt_and_scheduler()
visualise_scheduler(opt,
                    scheduler)

In [ ]:
def probabilistic_f1(labels, preds, beta=1):
    """
    Function taken from Awsaf's notebook:
    https://www.kaggle.com/code/awsaf49/rsna-bcd-efficientnet-tf-tpu-1vm-train
    """
    eps = 1e-5
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + eps)
    c_recall = ctp / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + eps)
        return result
    else:
        return 0.0

## Dataset Class - for loading in data

In [ ]:
def display_batch(batch, figsize=(16,10), n_cols=2, maximgs=None, cmap="bone"):
    if isinstance(batch, tuple) or isinstance(batch, list):
        if len(batch) == 3:
            imgs, targets, idx = batch
        else:
            imgs, targets, filenames, idx = batch
    else:
        imgs = batch
        targets = None
    #targets = targets.numpy().squeeze()
    
    n = imgs.shape[0] if not maximgs else min(imgs.shape[0], maximgs)
    n_rows = math.ceil(n / n_cols)
    
    fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=figsize)
    for i in range(n):
        x = i % n_cols
        y = i // n_cols
        ax=axs[y,x]
        im = imgs[i, :, :, :].squeeze()
        im = im.numpy().transpose((1,2,0)).squeeze()
        ax.imshow(im, cmap=cmap)
        if targets is not None:
            ax.set_title(f'Target {targets[i]}', fontsize=10)
        ax.set_xticks([]); ax.set_yticks([])
    fig.tight_layout()
    fig.show() 

In [ ]:
def pad_images_to_max_dim(batch, pad_value = 0):
    "Pad images with values if there are images of different size in batch"
    # consider non-symmetrical padding
    
    # print([img.shape for img in batch])
    max_h = max([img.shape[-2] for img in batch])
    max_w = max([img.shape[-1] for img in batch])
    # pad all images in batch
    padded_batch = []
    for i, img in enumerate(batch):
        pad_left, pad_right, pad_top, pad_bottom = 0, 0, 0, 0
        diff_w = max_w - img.shape[-1]
        diff_h = max_h - img.shape[-2]
        if diff_w > 0:
            pad_left = diff_w//2
            pad_right = diff_w - pad_left
        if diff_h > 0:
            pad_top = diff_h//2
            pad_bottom = diff_h - pad_top
        if any([pad_left, pad_right, pad_top, pad_bottom]):
            padded_img = torch.nn.functional.pad(img,
                            (pad_left, pad_right, pad_top, pad_bottom),
                            value=pad_value)
            padded_batch.append(padded_img)
        else:
            padded_batch.append(img)
        
    return padded_batch

def mixed_collate_imgs_fn_with_pad_value(batch, pad_value=0):
    batch_imgs = [x[0] for x in batch]
    
    padded_batch_imgs = pad_images_to_max_dim(batch_imgs, pad_value)
    padded_batch = []
    for b_item, pad_img in zip(batch, padded_batch_imgs):
        padded_batch.append( (pad_img, *b_item[1:]))
        
    return torch.utils.data.default_collate(padded_batch)

def mixed_collate_imgs_fn(batch):
    return mixed_collate_imgs_fn_with_pad_value(batch, Config.batch_pad_value)


def resolve_img_path(img_folder, patient_id, image_id, extension="png", has_patient_folder_sturcture=True):
    if has_patient_folder_sturcture:
        return os.path.join(img_folder, str(patient_id), f"{image_id}.{extension}")
    else:
        return os.path.join(img_folder, f"{image_id}.{extension}")

def load_img(img_path, resize_dim=None, resize_aspect_ratio=None, transform=None):    
    # need anydepth to load 16bit grayscale png
    # don't know if pngs are bgr or rgb
    img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
    if img is None:
        raise ValueError(f"No image {img_path} found")
    # cast to np float, as torch can't into uint16
    if resize_dim:
        img = cv2.resize(img, resize_dim)
    if resize_aspect_ratio and resize_aspect_ratio!=1.0:
        img = cv2.resize(img, tuple(int(d*resize_aspect_ratio) for d in img.shape[::-1]))
                
    # convert to RGB for pretrained networks
    # maybe we will move it to the different place
    if len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[0] == 1):
        # img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        pass
    else:
        img = cv2.cvtColor(img, cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
    
    # should we really normalize here? Why not in transforms?
    # again there are problems (there are?) with 16bits PNGS
    img = img.astype(np.float32)
    img_max = np.amax(img)
    img_min = np.amin(img)
    img_range = img_max-img_min
    if img_range > 0:
        img = (img-img_min)/img_range
    
    if transform:
        img = transform(image=img)['image']      
    
    # shouldn't be done in transforms?
    img = torch.tensor(img, dtype=torch.float)
    return img

# supports both 3-channels PNGs, and single-channel 16 bit PNG
class RSNAData(Dataset):
    def __init__(self, df,
                 img_folder,
                 resize_dim=None,
                 resize_aspect_ratio=None,
                 transform=None,
                 is_test=False,
                 has_patient_folder_sturcture=False,
                 extension="png",
                 return_filepath=False):
        
        assert not (resize_dim and resize_aspect_ratio)
        self.df = df
        self.is_test = is_test
        self.transform = transform
        self.img_folder = img_folder
        self.resize_dim = resize_dim
        self.resize_aspect_ratio=resize_aspect_ratio
        self.extension = extension
        self.has_patient_folder_sturcture=has_patient_folder_sturcture
        self.return_filepath=return_filepath
        
    def __getitem__(self, idx):
        row = self.df.loc[idx, :]

        img_path = resolve_img_path(self.img_folder, row["patient_id"], row["image_id"], self.extension, self.has_patient_folder_sturcture)
        img = load_img(img_path, self.resize_dim, self.resize_aspect_ratio, self.transform)
        
        if not self.is_test:
            target = self.df['cancer'][idx]
            target = torch.tensor(target, dtype=torch.float32)
            if self.return_filepath:
                return img, target, img_path, idx
            return (img, target, idx)
        if self.return_filepath:
            return img, img_path, idx
        return (img, idx)
    
    def __len__(self):
        return len(self.df)

class RSNAPatientTestData(Dataset):
    def __init__(self, df,
                 img_folder,
                 resize_dim=None,
                 resize_aspect_ratio=None,
                 transform=None,
                 has_patient_folder_sturcture=False,
                 extension="png"):
        
        assert not (resize_dim and resize_aspect_ratio)
        self.df = df
        self.grouped_df = df.groupby("prediction_id")
        self.grouped_df_indices = list(self.grouped_df.indices.keys())
        self.transform = transform
        self.img_folder = img_folder
        self.resize_dim = resize_dim
        self.resize_aspect_ratio=resize_aspect_ratio
        self.extension = extension
        self.has_patient_folder_sturcture=has_patient_folder_sturcture
        
    def __getitem__(self, idx):

        prediction_indices = self.grouped_df_indices[idx]
        rows = self.df.loc[prediction_indices, :]

        cc_view_row = rows[rows["view"]=="CC"]
        mlo_view_row = rows[rows["view"]=="MLO"]

        # should I check if both views are present?

        cc_img_path = resolve_img_path(self.img_folder, cc_view_row["patient_id"].values[0], cc_view_row["image_id"].values[0], self.extension, self.has_patient_folder_sturcture)
        mlo_img_path = resolve_img_path(self.img_folder, mlo_view_row["patient_id"].values[0], mlo_view_row["image_id"].values[0], self.extension, self.has_patient_folder_sturcture)

        cc_img = load_img(cc_img_path, self.resize_dim, self.resize_aspect_ratio, self.transform)
        mlo_img = load_img(mlo_img_path, self.resize_dim, self.resize_aspect_ratio, self.transform)


        return (cc_img, mlo_img, rows["prediction_id"].values[0])
    
    def __len__(self):
        return len(self.grouped_df_indices)

In [ ]:
# test pading
y= pad_images_to_max_dim([torch.tensor(np.random.rand(256,256)),
                          torch.tensor(np.random.randn(100,50))])
[x.shape for x in y]

In [ ]:
# Load the data and pass it onto the training function
df = pd.read_csv(Config.train_csv_path)
df['img_name'] = df['patient_id'].astype(str) + "/" + df['image_id'].astype(str) + ".png"
df = df.sample(frac=1).reset_index(drop=True)
df.head()

## Basic Image Augmentations

In [ ]:
# TODO, make dependent on timm.data.resolve_data_config(model)

@albumentations.augmentations.functional.preserve_channel_dim
def clahe_8or16(img, clip_limit=2.0, tile_grid_size=(8, 8)):
    clahe_mat = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = clahe_mat.apply(img)
    else:
        raise TypeError("clahe supports only grayscale inputs")

    return img

albumentations.augmentations.functional.clahe=clahe_8or16

class Normalize8or16bit(Normalize):

    def __init__(
        self,
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        always_apply=False,
        p=1.0,
    ):    
        super().__init__(mean, std, always_apply, p, 1.0)

    def apply(self, image, **params):
        return albumentations.augmentations.functional.normalize(image, self.mean, self.std, np.iinfo(image.dtype).max)

    def get_transform_init_args_names(self):
        return ("mean", "std")
    
    

class ToRGB(ImageOnlyTransform):
    def __init__(self, always_apply=True, p=1.0):
        super(ToRGB, self).__init__(always_apply=always_apply, p=p)

    def apply(self, img, **params):

        return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    def get_transform_init_args_names(self):
        return ()
    
class ExpandTo3Channels(ImageOnlyTransform):
    def __init__(self, always_apply=True, p=1.0):
        super().__init__(always_apply=always_apply, p=p)

    def apply(self, img, **params):
        return img.reshape(img.shape[0],img.shape[1],1)*np.ones(3).reshape(1,1,3)

    def get_transform_init_args_names(self):
        return ()    
    
# augmentations, for mammography
train_augments = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    Affine(
        scale=(0.9, 1.1),
        translate_percent=(-0.02, 0.02),
        rotate=(-10,10), p=0.5),
    RandomGamma(p=0.5),
    Normalize(mean=(0.5), std=(0.5), max_pixel_value=1.0, p=1.0),
    ExpandTo3Channels(p=1.0),
    ToTensorV2(p=1.0)
])

valid_augments = Compose([
    Normalize(mean=(0.5), std=(0.5), max_pixel_value=1.0, p=1.0),
    ExpandTo3Channels(p=1.0),
    ToTensorV2(p=1.0)
])

In [ ]:
class RSNAModel(pl.LightningModule):
    def __init__(self, 
                 model_name,
                 model_params,
                 pretrained,
                 lr,
                 warmup_lr,
                 warmup_epochs,
                 t_max,
                 min_lr,
                 weight_decay,
                 features_size=None,
                 pos_weight=None,   
                ):
        super().__init__()
        # save_hyperparameters() is used to specify which init arguments should 
        # be saved in the checkpoint file to be used to instantiate the model
        # from the checkpoint later.
        self.save_hyperparameters()
        self.model = timm.create_model(model_name, 
                               pretrained=pretrained,
                               **model_params
                              )
        classes_count = 1
        if not features_size:
            features_size = self.model(torch.randn(1, 3, 512, 512)).shape[-1]
        self.fc = nn.Linear(features_size, classes_count)
        
        # Loss functions
        # CHECKME - I believe that positive weight could not work here after re-loading model
        pos_weight=None if pos_weight is None else torch.tensor([pos_weight], dtype=torch.float32)
        self.train_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.valid_loss = nn.BCEWithLogitsLoss()

        # Metric
        self.roc_auc = torchmetrics.classification.BinaryAUROC()
        self.pr_curve = torchmetrics.classification.BinaryPrecisionRecallCurve()
        self.valid_stat_scores = torchmetrics.classification.BinaryStatScores()
        self.train_stat_scores = torchmetrics.classification.BinaryStatScores()
        self.binary_f1 = torchmetrics.F1Score(task='binary')
    
    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr,
                               weight_decay=self.hparams.weight_decay)

        scheduler1 = torch.optim.lr_scheduler.ConstantLR(opt,
                                                         factor=self.hparams.warmup_lr/self.hparams.lr,
                                                         total_iters=self.hparams.warmup_epochs)
        scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(
                        opt, 
                        T_max=self.hparams.t_max,
                        eta_min=self.hparams.min_lr
                     )
        sch = torch.optim.lr_scheduler.SequentialLR(opt,
                                                    schedulers=[scheduler1, scheduler2],
                                                    milestones=[self.hparams.warmup_epochs])
        
        return [opt], [sch]
        
    def forward(self, x):
        features = self.model(x)
        output = self.fc(features)
        return output

    def predict(self, x):
        return torch.sigmoid(self(x))
    
    def training_step(self, batch, batch_idx):
        imgs = batch[0]
        target = batch[1].unsqueeze(-1)
        
        out = self(imgs)
        positive_outcomes_mean = out[out>=0].mean()
        negative_outcomes_mean = out[out<0].mean()
        cancer_percent = target[target>0].shape[0]/target.shape[0]
        
        predictions = F.sigmoid(out)
        
        # shall we compute every x epochs? (but we will loose epoch metrics!)
        true_positives, false_positives, true_negatives, false_negatives, sup = self.train_stat_scores(predictions, target)
        train_loss = self.train_loss(out, target)
        to_log = {'train/loss': train_loss.item(),
                  'train/positive_outcomes_mean': positive_outcomes_mean.item(),
                  'train/negative_outcomes_mean': negative_outcomes_mean.item(),
                  'train/accuracy': ((true_positives+true_negatives)/(true_positives+false_positives+true_negatives+false_negatives)).item(),
                  'train/precision': (true_positives/(true_positives+false_positives)).item(),
                  'train/recall': (true_positives/(true_positives+false_negatives)).item()
                 }
        self.log_dict(to_log)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        imgs = batch[0]
        target = batch[1].unsqueeze(-1)
        
        out = self(imgs)
        
        valid_loss = self.valid_loss(out, target)
        
        predictions = F.sigmoid(out)
          
        f1_current = self.binary_f1(predictions, target)
        self.roc_auc(predictions, target)
        self.valid_stat_scores(predictions, target)
        self.log_dict({"valid/loss": valid_loss.item(),
                       "valid/f1": f1_current.item()})     
        return valid_loss
    
    def training_epoch_end(self, outputs):
        true_positives, false_positives, true_negatives, false_negatives, sup = self.train_stat_scores.compute()
        to_log = {
                  'train/precision_epoch': (true_positives/(true_positives+false_negatives)).item(),
                  'train/recall_epoch': (true_positives/(true_positives+false_negatives)).item(),
                  'train/specificity_epoch': (true_negatives/(true_negatives+false_positives)).item()}
        self.log_dict(to_log) 
        self.train_stat_scores.reset()
    
    def validation_epoch_end(self, outputs):
        true_positives, false_positives, true_negatives, false_negatives, sup = self.valid_stat_scores.compute()
        to_log = {'valid/roc_auc_epoch': self.roc_auc.compute().item(),
                  'valid/f1_epoch': self.binary_f1.compute().item(),
                  'valid/precision_epoch': (true_positives/(true_positives+false_positives)).item(),
                  'valid/recall_epoch': (true_positives/(true_positives+false_negatives)).item(),
                  'valid/specificity_epoch': (true_negatives/(true_negatives+false_positives)).item()}
        self.log_dict(to_log) 
        self.binary_f1.reset()
        self.roc_auc.reset()
        self.valid_stat_scores.reset()
        
    # def test_step(self, batch, batch_idx):
    #    loss, acc = self._shared_eval_step(batch, batch_idx)
    #    metrics = {"test_acc": acc, "test_loss": loss}
    #    self.log_dict(metrics)
    #    return metrics
    


config_hparams_dict = dict(
    model_name=Config.model_name,
    lr=Config.lr,
    warmup_lr=Config.warmup_lr,
    warmup_epochs=Config.warmup_epochs,
    t_max=Config.t_max,
    min_lr=Config.min_lr,
    weight_decay=Config.weight_decay,
    pos_weight=Config.pos_weight)

In [ ]:
try:
    tb_logger = TensorBoardLogger(save_dir="tb_logs", name="rsna-breast-model")
except ModuleNotFoundError:
    tb_logger = None

loggers = [x for x in [wandb_logger, tb_logger] if x is not None]



checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    monitor="valid/f1_epoch",
    filename="ckpt_epoch={epoch:02d}_validf1={valid/f1_epoch:02.0f}",
    auto_insert_metric_name=False,
    save_top_k=3,
    mode="max",
    save_last=True,
    every_n_epochs=1)

lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [ ]:
def resolve_binary_class_upsample_weight(targets, upsample_positive_to_percent):
    "Returns weights for array of target classes so that positive examples are upsampled to given percent"
    
    weights = targets.astype(np.float64)
    
    dataset_len = len(weights)
    positives_len = len(targets[targets > 0])
    negatives_len = dataset_len-positives_len

    if positives_len == 0:
        return np.ones_like(targets, dtype=np.float64)
    
    x = negatives_len*upsample_positive_to_percent / (positives_len -  upsample_positive_to_percent*positives_len)
    weights[targets > 0] = x
    weights[targets <= 0 ] = 1
    return weights

## Main loop

In [ ]:


# add stratified k folds when ready
kfold = sklearn.model_selection.StratifiedShuffleSplit(
    n_splits=Config.splits_count, train_size=0.8)
for fold_, (train_idx, valid_idx) in enumerate(kfold.split(df, df['cancer'].values)):
    print(f"{'='*40} Fold: {fold_} / {Config.splits_count-1} {'='*40}")


    train_df = df.loc[train_idx].reset_index(drop=True).copy()
    valid_df = df.loc[valid_idx].reset_index(drop=True).copy()
        
    if Config.debug:
        train_df = train_df.iloc[:int(len(train_df)*Config.debug_data_use_only_percent), :]
        valid_df = train_df.iloc[:int(len(valid_df)*Config.debug_data_use_only_percent), :]
    
    print(f"Train df len {len(train_df)}, cancers {train_df['cancer'].sum()}, percent {train_df['cancer'].sum()/len(train_df)}")
    print(f"Valid df len {len(valid_df)}, cancers {valid_df['cancer'].sum()}, percent {valid_df['cancer'].sum()/len(valid_df)}")        
    
    train_dataset = RSNAData(
        df = train_df,
        img_folder = Config.train_imgs_path,
        has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
        resize_aspect_ratio = Config.resize_aspect_ratio,
        transform=train_augments
    )
    valid_dataset = RSNAData(
        df = valid_df,
        img_folder = Config.train_imgs_path,
        has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
        resize_aspect_ratio = Config.resize_aspect_ratio,
        transform = valid_augments
    )
    
    if Config.weight_random_sampler:
        train_rows_weight = resolve_binary_class_upsample_weight(train_df["cancer"].values, Config.positive_upsample_to_percent)   
        print(f"In train dataset cancer will be upsampled to percent {train_rows_weight[train_rows_weight>1].sum()/train_rows_weight.sum()}")
        train_sampler = torch.utils.data.sampler.WeightedRandomSampler(train_rows_weight, len(train_df))
        shuffle=False
    else:
        train_sampler=None
        shuffle=True
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.train_bs,
        shuffle=shuffle,
        sampler=train_sampler,
        num_workers=Config.dataloader_workers_count,
        collate_fn=mixed_collate_imgs_fn,
        pin_memory=True
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=Config.train_bs,
        shuffle=False,
        num_workers=Config.dataloader_workers_count,
        collate_fn=mixed_collate_imgs_fn,
        pin_memory=True
    )
    # CHECKME after introducing k-fold start with fresh copy of base model
    
    if checkpoint_path:
        if Config.overwrite_checkpoint_hparams:
            hparams = config_hparams_dict
        else:
            hparams = {}
        # don't download pretrained weights for checkpointed model
        hparams["pretrained"]=False
        model = RSNAModel.load_from_checkpoint(checkpoint_path, 
                                              **hparams)
    else:   
        model = RSNAModel(pretrained=Config.use_pretrained,
                          model_params=Config.model_params,
                          **config_hparams_dict)
    
    if WandbConfig.watch:
        wandb.watch(model,
                    log=WandbConfig.watch,
                    log_freq=WandbConfig.watch_log_freq)
    
    trainer = pl.Trainer(
        max_epochs=Config.epochs_count,
        accelerator=Config.accelerator,
        devices=Config.num_devices,
        precision=Config.precision,
        logger = loggers,
        callbacks=[checkpoint_callback, lr_monitor]
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        trainer.fit(model, train_loader, valid_loader)
    wandb.finish()

In [ ]:
train_dataset = RSNAData(
    df = df,
    img_folder = Config.train_imgs_path,
    has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
    resize_aspect_ratio = Config.resize_aspect_ratio,
    transform=train_augments
)

train_loader = DataLoader(
    train_dataset,
    batch_size=Config.train_bs,
    shuffle=True,
    num_workers=0,
    collate_fn=mixed_collate_imgs_fn
)
minibatch = next(iter(train_loader))
print(minibatch[0].shape)
print(minibatch[0].mean((1,2,3)))
print(minibatch[0].min())
print(minibatch[0].max())
minibatch[0] += 1.0
minibatch[0] /= 2.0
print(minibatch[0].mean((1,2,3)))

display_batch(minibatch, maximgs=4)

In [ ]:
x_dataset = RSNAData(
    df = df,
    img_folder = Config.train_imgs_path,
    has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
    # img_folder = "/kaggle/input/rsna-breast-png-roi/train_images/256/",
    #img_folder = "/kaggle/input/rsna-cut-off-empty-space-from-images",
    transform=Compose([ToRGB(), ToTensorV2()]),
    return_filepath=True
)

weighted_x_loader = DataLoader(
    x_dataset,
    batch_size=Config.train_bs,
    sampler=torch.utils.data.sampler.WeightedRandomSampler(
        resolve_binary_class_upsample_weight(df["cancer"].values, Config.positive_upsample_to_percent),
        len(df),
        replacement=False),
    num_workers=0,
    collate_fn=mixed_collate_imgs_fn,
)

#if Config.debug or True:
#    for minibatch in weighted_x_loader:
#        imgs = minibatch[0]
#        if torch.isnan(minibatch[0]).sum() > 0:
#            raise ValueError("bad pixel values")
#        if torch.isnan(minibatch[1]).sum() > 0:
#            raise ValueError("bad targets values")

In [ ]:
minibatch = next(iter(weighted_x_loader))
display_batch(minibatch, maximgs=10,figsize=(32,32))
minibatch[1]

In [ ]:
v_dataset = RSNAData(
    df = df,
    img_folder = Config.train_imgs_path,
    has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
    transform = valid_augments
)
v_loader = DataLoader(
    v_dataset,
    batch_size=Config.train_bs,
    shuffle=False,
    num_workers=0,
    collate_fn=mixed_collate_imgs_fn,
    pin_memory=True
)
minibatch = next(iter(v_loader))
print(minibatch[0].shape)
print(minibatch[0].mean((1,2,3)))
print(minibatch[0].min())
print(minibatch[0].max())
print(torch.isnan(minibatch[0]).sum())
minibatch[0] += 1.0
minibatch[0] /= 2.0
print(minibatch[0].mean((1,2,3)))
    
display_batch(minibatch, maximgs=20,figsize=(32,32))

minibatch[1]


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=tb_logs/

In [ ]:
test_df = pd.read_csv(Config.test_csv_path)
test_df['img_name'] = test_df['patient_id'].astype(str) + "/" + test_df['image_id'].astype(str) + ".png"

test_dataset = RSNAData(
    df = test_df,
    img_folder = Config.test_imgs_path,
    has_patient_folder_sturcture = Config.data_has_patient_folder_sturcture,
    resize_aspect_ratio = Config.resize_aspect_ratio,
    is_test=True,
    transform = valid_augments
)

test_loader = DataLoader( 
    test_dataset,
    batch_size=Config.train_bs,
    shuffle=False,
    num_workers=Config.dataloader_workers_count,
    collate_fn=mixed_collate_imgs_fn,
    pin_memory=True
)



In [ ]:
trainer.checkpoint_callback.best_model_path

In [ ]:
best_checkpoints = trainer.checkpoint_callback.best_model_path
eval_model = RSNAModel.load_from_checkpoint(checkpoint_path = best_checkpoints, pretrained=False)
eval_model = eval_model.to(device)
eval_model.eval()
eval_model.freeze()

predictions = []
indices = []

with torch.no_grad():
    for batch_imgs, batch_indices in test_loader:
        cancer_predictions = eval_model.predict(batch_imgs.to(device))
        predictions.extend(cancer_predictions.cpu().detach().numpy().flatten().tolist())
        indices.extend(batch_indices.cpu().detach().numpy().tolist())

summary_df = pd.DataFrame({"prediction_id": test_df["prediction_id"].values[indices], "cancer": predictions})
submission_series = summary_df.groupby(["prediction_id"])["cancer"].max()
submission_series.to_csv("submission.csv")